In [1]:
# all imports up here, rerun this block when adding
import pandas as pd
import numpy as np
import cv2 as cv

In [2]:
#assumes dataset in folder
wines   = pd.read_csv("XWines_Full_100K_wines.csv", low_memory=False, encoding="utf-8", memory_map=True)
ratings = pd.read_csv("XWines_Full_21M_ratings.csv", low_memory=False, encoding="utf-8", memory_map=True)
len(wines), len(ratings)

print(ratings)
print(wines)

          RatingID   UserID  WineID Vintage  Rating                 Date
0                1  1604441  136103    1950     4.0  2019-10-14 11:20:52
1                2  1291483  136103    1950     5.0  2019-11-28 03:36:33
2                3  1070605  104036    1950     5.0  2017-12-28 10:15:55
3                4  1080181  144864    1950     5.0  2016-06-23 02:16:22
4                5  1834379  111430    1950     5.0  2021-05-16 17:58:14
...            ...      ...     ...     ...     ...                  ...
21013531  21013532  2015383  113302    N.V.     3.0  2019-02-16 14:15:48
21013532  21013533  1868739  111440    N.V.     2.0  2018-09-30 16:47:05
21013533  21013534  1402947  142467    N.V.     3.0  2021-01-29 19:21:14
21013534  21013535  1360350  111440    N.V.     4.0  2021-07-26 14:02:14
21013535  21013536  1192603  111393    N.V.     5.0  2016-11-17 04:48:43

[21013536 rows x 6 columns]
        WineID                           WineName       Type  \
0       100001                 

In [3]:
def read():
    ##assumes dataset in folder
    wines   = pd.read_csv("XWines_Full_100K_wines.csv", low_memory=False, encoding="utf-8", memory_map=True)
    ratings = pd.read_csv("XWines_Full_21M_ratings.csv", low_memory=False, encoding="utf-8", memory_map=True)
    return wines,ratings

In [21]:

wine_type = input("Enter the type of wine (e.g., red, white, rosé, sparkling): ").lower()
wine_country = input("Enter the country of the wine (e.g., Brazil, New Zealand): ").lower()
wine_body = input("Enter the body of the wine(e.g. ligth-bodied, medium-bodied, full-bodied): ").lower()

filtered_wines = wines.copy()

filtered_wines = wines[(wines['Type'].str.lower() == wine_type) 
                       & (wines['Country'].str.lower() == wine_country)
                       & (wines['Body'].str.lower() == wine_body)]

if filtered_wines.empty:
    filtered_wines = wines[(wines['Type'].str.lower() == wine_type)
                           & (wines['Country'].str.lower() == wine_country)]
    if not filtered_wines.empty:
        print(f"No '{wine_type}' wines in country '{wine_country}' found with body '{wine_body}'")
        print(f"Showing all '{wine_type}' wines from country '{wine_country}' instead.")
    else:
        filtered_wines = wines[wines['Type'].str.lower() == wine_type]
        if not filtered_wines.empty:
            print(f"No '{wine_type}' wines in country '{wine_country}'. Showing all '{wine_type}'instead")
        else:
            print(f"No wines found for the type '{wine_type}'.")
else:
    print(f"Showing '{wine_type}' wines from '{wine_country}'.")

if not filtered_wines.empty:
    print(filtered_wines)
else:
    print("No wines found that match your preferences.")

Showing 'white' wines from 'new zealand'.
        WineID              WineName   Type      Elaborate  \
93417   193546            Chardonnay  White  Varietal/100%   
93434   193563            Chardonnay  White  Varietal/100%   
93521   193650    Village Chardonnay  White  Varietal/100%   
93542   193671     Estate Chardonnay  White  Varietal/100%   
93578   193707            Chardonnay  White  Varietal/100%   
...        ...                   ...    ...            ...   
100459  200595        Gewürztraminer  White  Varietal/100%   
100561  200705    Unoaked Chardonnay  White  Varietal/100%   
100586  200732     Helios Chardonnay  White  Varietal/100%   
100593  200741   Chardonnay Dolly SP  White  Varietal/100%   
100635  200785  Chalk Ridge Viognier  White  Varietal/100%   

                    Grapes                                          Harmonize  \
93417       ['Chardonnay']     ['Pork', 'Rich Fish', 'Vegetarian', 'Poultry']   
93434       ['Chardonnay']     ['Pork', 'Rich Fish'

In [22]:
filtered_reviews = ratings[ratings['WineID'].isin(filtered_wines['WineID'])]

# Merge filtered_wines and filtered_reviews on 'WineID'
merged_df = pd.merge(filtered_wines, filtered_reviews, on='WineID', how='inner')

# Identify columns that belong to the wines and ratings DataFrames
wine_columns = ['WineID', 'Type', 'Country', 'Body']  
rating_columns = ['RatingID', 'WineID', 'UserID', 'Rating']  

# Split the merged DataFrame back into wines and ratings
filtered_wines = merged_df[wine_columns].drop_duplicates()  # Drop duplicates to get unique wine data
filtered_reviews = merged_df[rating_columns]

print(filtered_wines.head(10))
print(filtered_reviews.head(10))


      WineID   Type      Country         Body
0     193546  White  New Zealand  Full-bodied
1271  193563  White  New Zealand  Full-bodied
2363  193650  White  New Zealand  Full-bodied
3025  193671  White  New Zealand  Full-bodied
3643  193707  White  New Zealand  Full-bodied
4176  193723  White  New Zealand  Full-bodied
4684  193755  White  New Zealand  Full-bodied
5142  193907  White  New Zealand  Full-bodied
5479  193954  White  New Zealand  Full-bodied
5806  193970  White  New Zealand  Full-bodied
   RatingID  WineID   UserID  Rating
0     98334  193546  1138214     4.5
1    303746  193546  1214823     2.0
2    337623  193546  1279122     5.0
3    341880  193546  1152696     3.5
4    386200  193546  1210299     5.0
5    432851  193546  1835021     3.0
6    465509  193546  1215365     4.0
7    473655  193546  1770616     4.0
8    491310  193546  1263603     4.5
9    509211  193546  1295910     3.0


In [23]:
from CB import ContentBasedRecommender

print(ContentBasedRecommender)

recommender = ContentBasedRecommender(filtered_reviews, filtered_wines)
recommender.fit()
# choose a userID from the above filtered_reviews df
recommendations = recommender.recommend(1138214, top_n=5)
print(recommendations)


<class 'CB.ContentBasedRecommender'>
[193563, 193650, 193671, 193707, 193723]
